# 1 - Finetuning choice CNN on the LFW dataset

boilerplate processing code taken from: https://www.kaggle.com/code/tanapolnuatho/classification-with-cosine-similarity-loss

In [10]:
import os, time
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Subset, Dataset
import torchvision
from torchvision import transforms, models
import tarfile
from sklearn.model_selection import train_test_split

In [3]:
TGZ_PATH = "/Users/02matt/MASc/Model-Inversion/archive/lfw-funneled.tgz"
DATA_DIR = "/Users/02matt/MASc/Model-Inversion/data"

os.makedirs(DATA_DIR, exist_ok=True)

if os.path.exists(TGZ_PATH):
    try:
        with tarfile.open(TGZ_PATH, "r:gz") as tar:
            tar.extractall(path=DATA_DIR)
    except Exception as e:
        print(f"Error {e}")
else:
    print("Unable to locate file")

In [5]:
DATA_DIR = os.path.join(DATA_DIR, 'lfw_funneled')

Go with classes that have 10 or more examples

In [7]:
MIN_IMAGES_PER_CLASS = 10
all_people = os.listdir(DATA_DIR)
valid_people = []

for person in all_people:
    person_path = os.path.join(DATA_DIR, person)
    if os.path.isdir(person_path):
        num_imgs = len([f for f in os.listdir(person_path) if f.endswith(('.jpg', '.png'))])
        if num_imgs >= MIN_IMAGES_PER_CLASS:
            valid_people.append(person)


In [9]:
class_to_idx = {name: i for i, name in enumerate(valid_people)}
idx_to_class = {i: name for name, i in class_to_idx.items()}

all_image_paths = []
all_labels = []

for person in valid_people:
    person_path = os.path.join(DATA_DIR, person)
    for img_name in os.listdir(person_path):
        if img_name.endswith(('.jpg', '.png')):
            all_image_paths.append(os.path.join(person_path, img_name))
            all_labels.append(class_to_idx[person])

if len(all_labels) > 0:
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_image_paths, all_labels, test_size=0.2, stratify=all_labels, random_state=42
    )
    print(f"Train images: {len(train_paths)}")
    print(f"Test images: {len(test_paths)}")
else:
    print("Error: No images found! Check DATA_DIR path again.")

Train images: 3459
Test images: 865


Need to define own LFW dataset class since the one on pytorch has essentially been deprecated.

In [11]:
class LFWDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

Training Hyperparameters

In [14]:
BATCH_SIZE = 32
EMBEDDING_SIZE = 512
NUM_EPOCHS = 20 
LEARNING_RATE = 0.001

Using `ResNet18` from torchvision.

In [15]:
weights = models.ResNet18_Weights.DEFAULT  # ImageNet-pretrained
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# Train-time augmentation (to 224x224) + ImageNet normalization
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


In [13]:
train_set = LFWDataset(image_paths=train_paths, labels=train_labels, transform=train_transforms)
val_set = LFWDataset(image_paths=test_paths, labels=test_labels, transform=val_transforms)

In [ ]:
train_loader = DataLoader(dataset=train_set, batch_size=)